In [1]:
import pandas as pd, numpy as np, time
from sklearn.model_selection import train_test_split, GridSearchCV

In [19]:
data = pd.read_csv('flight\\flight_delays_train.csv')
test_df = pd.read_csv('flight\\flight_delays_test.csv')

In [20]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


In [21]:
y_train = data['dep_delayed_15min'].map({'Y': 1, 'N': 0}).astype('int').values
data['Route']=data['Dest']+data['Origin']

In [22]:
cols = ["UniqueCarrier","Dest","Origin", "Month", "DayOfWeek", "DayofMonth", "Route"]
for item in cols:
    data[item] = data[item].astype("category").cat.codes+1
    test_df[item] = data[item].astype("category").cat.codes+1

In [23]:
train, test, y_train, y_test = train_test_split(data.drop(["dep_delayed_15min"], axis=1), y_train,
                                                random_state=17, test_size=0.25)

In [24]:
import xgboost as xgb
from sklearn import metrics

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))
model = xgb.XGBClassifier()

In [18]:
# Parameter Tuning

param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
              "n_estimators": [200],
              "learning_rate": [0.05, 0.1,0.16],}
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(train, y_train)

grid_search.best_estimator_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 15.1min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 22.8min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 30.0min finished


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=10, min_child_weight=3, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [20]:
grid_search.best_estimator_, grid_search.best_score_

(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
        max_depth=10, min_child_weight=3, missing=None, n_estimators=200,
        n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
        reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
        silent=True, subsample=1), 0.8206533333333333)

In [25]:
model = xgb.XGBClassifier(max_depth=10, min_child_weight=3,  n_estimators=200,\
                          n_jobs=-1 , verbose=1,learning_rate=0.05)
model.fit(train,y_train)

auc(model, train, test)

(0.9246007591005135, 0.7407616852966057)

In [26]:
def write_to_submission_file(predicted_labels, out_file,
                             target='dep_delayed_15min', index_label="id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(0, predicted_labels.shape[0]),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)
y_pred = model.predict(test_df)
write_to_submission_file(y_pred, 'subm2.csv')

In [27]:
import catboost as cb
cat_features_index = [0,1,2,3,4,5,6]
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
cb_c = cb.CatBoostClassifier()
cb_model = GridSearchCV(cb_c, params, scoring="roc_auc", cv = 3)


In [28]:
cb_model.fit(train, y_train)
#With Categorical features



0:	learn: 0.6774172	total: 129ms	remaining: 38.7s
1:	learn: 0.6625117	total: 202ms	remaining: 30s
2:	learn: 0.6485879	total: 250ms	remaining: 24.7s
3:	learn: 0.6359855	total: 302ms	remaining: 22.3s
4:	learn: 0.6241353	total: 368ms	remaining: 21.7s
5:	learn: 0.6130021	total: 445ms	remaining: 21.8s
6:	learn: 0.6026523	total: 505ms	remaining: 21.1s
7:	learn: 0.5926799	total: 576ms	remaining: 21s
8:	learn: 0.5838256	total: 658ms	remaining: 21.3s
9:	learn: 0.5756554	total: 726ms	remaining: 21s
10:	learn: 0.5674942	total: 798ms	remaining: 21s
11:	learn: 0.5596419	total: 865ms	remaining: 20.7s
12:	learn: 0.5532104	total: 949ms	remaining: 20.9s
13:	learn: 0.5465902	total: 1.01s	remaining: 20.7s
14:	learn: 0.5409849	total: 1.08s	remaining: 20.6s
15:	learn: 0.5350559	total: 1.15s	remaining: 20.5s
16:	learn: 0.5298250	total: 1.2s	remaining: 20s
17:	learn: 0.5252625	total: 1.26s	remaining: 19.7s
18:	learn: 0.5200764	total: 1.31s	remaining: 19.5s
19:	learn: 0.5158897	total: 1.36s	remaining: 19.1s
2

161:	learn: 0.4414207	total: 10s	remaining: 8.54s
162:	learn: 0.4413867	total: 10.1s	remaining: 8.48s
163:	learn: 0.4413495	total: 10.1s	remaining: 8.41s
164:	learn: 0.4412912	total: 10.2s	remaining: 8.34s
165:	learn: 0.4412427	total: 10.3s	remaining: 8.31s
166:	learn: 0.4411760	total: 10.4s	remaining: 8.26s
167:	learn: 0.4411221	total: 10.5s	remaining: 8.21s
168:	learn: 0.4410758	total: 10.5s	remaining: 8.15s
169:	learn: 0.4410159	total: 10.6s	remaining: 8.08s
170:	learn: 0.4409490	total: 10.6s	remaining: 8.01s
171:	learn: 0.4409060	total: 10.7s	remaining: 7.94s
172:	learn: 0.4408740	total: 10.7s	remaining: 7.88s
173:	learn: 0.4408118	total: 10.8s	remaining: 7.81s
174:	learn: 0.4407668	total: 10.8s	remaining: 7.74s
175:	learn: 0.4407413	total: 10.9s	remaining: 7.67s
176:	learn: 0.4407189	total: 10.9s	remaining: 7.6s
177:	learn: 0.4406862	total: 11s	remaining: 7.53s
178:	learn: 0.4406551	total: 11s	remaining: 7.46s
179:	learn: 0.4406119	total: 11.1s	remaining: 7.39s
180:	learn: 0.44058

27:	learn: 0.4920673	total: 822ms	remaining: 7.98s
28:	learn: 0.4898615	total: 846ms	remaining: 7.91s
29:	learn: 0.4874253	total: 868ms	remaining: 7.81s
30:	learn: 0.4855865	total: 892ms	remaining: 7.74s
31:	learn: 0.4838282	total: 913ms	remaining: 7.65s
32:	learn: 0.4824220	total: 935ms	remaining: 7.56s
33:	learn: 0.4808304	total: 956ms	remaining: 7.48s
34:	learn: 0.4788853	total: 978ms	remaining: 7.41s
35:	learn: 0.4774216	total: 1.01s	remaining: 7.39s
36:	learn: 0.4761118	total: 1.03s	remaining: 7.36s
37:	learn: 0.4746308	total: 1.06s	remaining: 7.29s
38:	learn: 0.4732026	total: 1.08s	remaining: 7.21s
39:	learn: 0.4720495	total: 1.1s	remaining: 7.15s
40:	learn: 0.4707746	total: 1.13s	remaining: 7.11s
41:	learn: 0.4695839	total: 1.15s	remaining: 7.04s
42:	learn: 0.4685675	total: 1.17s	remaining: 6.97s
43:	learn: 0.4676343	total: 1.19s	remaining: 6.95s
44:	learn: 0.4668394	total: 1.22s	remaining: 6.89s
45:	learn: 0.4660929	total: 1.24s	remaining: 6.83s
46:	learn: 0.4650861	total: 1.26

190:	learn: 0.4417439	total: 4.59s	remaining: 2.62s
191:	learn: 0.4417207	total: 4.62s	remaining: 2.6s
192:	learn: 0.4416681	total: 4.64s	remaining: 2.57s
193:	learn: 0.4416401	total: 4.66s	remaining: 2.55s
194:	learn: 0.4416103	total: 4.68s	remaining: 2.52s
195:	learn: 0.4415870	total: 4.7s	remaining: 2.49s
196:	learn: 0.4415616	total: 4.72s	remaining: 2.47s
197:	learn: 0.4415251	total: 4.75s	remaining: 2.44s
198:	learn: 0.4414869	total: 4.77s	remaining: 2.42s
199:	learn: 0.4414506	total: 4.79s	remaining: 2.4s
200:	learn: 0.4414165	total: 4.82s	remaining: 2.37s
201:	learn: 0.4413842	total: 4.84s	remaining: 2.35s
202:	learn: 0.4413062	total: 4.86s	remaining: 2.32s
203:	learn: 0.4412459	total: 4.88s	remaining: 2.29s
204:	learn: 0.4412053	total: 4.9s	remaining: 2.27s
205:	learn: 0.4411525	total: 4.92s	remaining: 2.25s
206:	learn: 0.4411229	total: 4.94s	remaining: 2.22s
207:	learn: 0.4410866	total: 4.98s	remaining: 2.2s
208:	learn: 0.4410556	total: 5s	remaining: 2.18s
209:	learn: 0.441023

52:	learn: 0.4605271	total: 1.37s	remaining: 6.38s
53:	learn: 0.4598794	total: 1.4s	remaining: 6.36s
54:	learn: 0.4593145	total: 1.42s	remaining: 6.32s
55:	learn: 0.4587924	total: 1.44s	remaining: 6.28s
56:	learn: 0.4581348	total: 1.46s	remaining: 6.24s
57:	learn: 0.4576725	total: 1.49s	remaining: 6.2s
58:	learn: 0.4570929	total: 1.51s	remaining: 6.16s
59:	learn: 0.4565587	total: 1.53s	remaining: 6.11s
60:	learn: 0.4562434	total: 1.56s	remaining: 6.1s
61:	learn: 0.4557283	total: 1.58s	remaining: 6.07s
62:	learn: 0.4553860	total: 1.6s	remaining: 6.03s
63:	learn: 0.4550764	total: 1.63s	remaining: 5.99s
64:	learn: 0.4547819	total: 1.65s	remaining: 5.96s
65:	learn: 0.4542813	total: 1.67s	remaining: 5.92s
66:	learn: 0.4540680	total: 1.69s	remaining: 5.88s
67:	learn: 0.4537426	total: 1.71s	remaining: 5.85s
68:	learn: 0.4534157	total: 1.74s	remaining: 5.84s
69:	learn: 0.4529889	total: 1.77s	remaining: 5.8s
70:	learn: 0.4527876	total: 1.79s	remaining: 5.77s
71:	learn: 0.4524870	total: 1.81s	re

...

63:	learn: 0.3908558	total: 5.54s	remaining: 20.4s
64:	learn: 0.3896361	total: 5.64s	remaining: 20.4s
65:	learn: 0.3889532	total: 5.72s	remaining: 20.3s
66:	learn: 0.3881649	total: 5.81s	remaining: 20.2s
67:	learn: 0.3871339	total: 5.9s	remaining: 20.1s
68:	learn: 0.3862713	total: 5.99s	remaining: 20s
69:	learn: 0.3859142	total: 6.07s	remaining: 19.9s
70:	learn: 0.3853532	total: 6.15s	remaining: 19.8s
71:	learn: 0.3847086	total: 6.23s	remaining: 19.7s
72:	learn: 0.3839980	total: 6.32s	remaining: 19.7s
73:	learn: 0.3832973	total: 6.42s	remaining: 19.6s
74:	learn: 0.3822734	total: 6.5s	remaining: 19.5s
75:	learn: 0.3819017	total: 6.59s	remaining: 19.4s
76:	learn: 0.3810627	total: 6.67s	remaining: 19.3s
77:	learn: 0.3801769	total: 6.76s	remaining: 19.2s
78:	learn: 0.3795471	total: 6.85s	remaining: 19.2s
79:	learn: 0.3790797	total: 6.93s	remaining: 19.1s
80:	learn: 0.3783774	total: 7.02s	remaining: 19s
81:	learn: 0.3775583	total: 7.11s	remaining: 18.9s
82:	learn: 0.3767856	total: 7.2s	rema

223:	learn: 0.2961573	total: 19.7s	remaining: 6.67s
224:	learn: 0.2959283	total: 19.7s	remaining: 6.58s
225:	learn: 0.2954628	total: 19.8s	remaining: 6.49s
226:	learn: 0.2950885	total: 19.9s	remaining: 6.41s
227:	learn: 0.2946019	total: 20s	remaining: 6.32s
228:	learn: 0.2942250	total: 20.1s	remaining: 6.23s
229:	learn: 0.2938330	total: 20.2s	remaining: 6.14s
230:	learn: 0.2931862	total: 20.3s	remaining: 6.06s
231:	learn: 0.2926551	total: 20.4s	remaining: 5.97s
232:	learn: 0.2922168	total: 20.5s	remaining: 5.88s
233:	learn: 0.2918414	total: 20.5s	remaining: 5.79s
234:	learn: 0.2914863	total: 20.6s	remaining: 5.7s
235:	learn: 0.2912119	total: 20.7s	remaining: 5.62s
236:	learn: 0.2908166	total: 20.8s	remaining: 5.53s
237:	learn: 0.2902706	total: 20.9s	remaining: 5.44s
238:	learn: 0.2897715	total: 21s	remaining: 5.35s
239:	learn: 0.2894800	total: 21.1s	remaining: 5.27s
240:	learn: 0.2889101	total: 21.2s	remaining: 5.18s
241:	learn: 0.2883899	total: 21.2s	remaining: 5.09s
242:	learn: 0.287

88:	learn: 0.4278604	total: 3.88s	remaining: 9.21s
89:	learn: 0.4276671	total: 3.92s	remaining: 9.16s
90:	learn: 0.4275540	total: 3.97s	remaining: 9.12s
91:	learn: 0.4274138	total: 4.01s	remaining: 9.07s
92:	learn: 0.4272800	total: 4.06s	remaining: 9.03s
93:	learn: 0.4271001	total: 4.1s	remaining: 8.99s
94:	learn: 0.4269430	total: 4.14s	remaining: 8.94s
95:	learn: 0.4267639	total: 4.19s	remaining: 8.9s
96:	learn: 0.4266269	total: 4.23s	remaining: 8.85s
97:	learn: 0.4264499	total: 4.27s	remaining: 8.81s
98:	learn: 0.4262850	total: 4.32s	remaining: 8.76s
99:	learn: 0.4261510	total: 4.36s	remaining: 8.72s
100:	learn: 0.4260198	total: 4.41s	remaining: 8.68s
101:	learn: 0.4258732	total: 4.48s	remaining: 8.69s
102:	learn: 0.4256810	total: 4.53s	remaining: 8.66s
103:	learn: 0.4255687	total: 4.57s	remaining: 8.62s
104:	learn: 0.4253937	total: 4.62s	remaining: 8.57s
105:	learn: 0.4252113	total: 4.66s	remaining: 8.53s
106:	learn: 0.4250010	total: 4.7s	remaining: 8.48s
107:	learn: 0.4248186	total

247:	learn: 0.4070192	total: 11.2s	remaining: 2.34s
248:	learn: 0.4069274	total: 11.2s	remaining: 2.3s
249:	learn: 0.4068167	total: 11.3s	remaining: 2.25s
250:	learn: 0.4067147	total: 11.3s	remaining: 2.21s
251:	learn: 0.4066248	total: 11.3s	remaining: 2.16s
252:	learn: 0.4065262	total: 11.4s	remaining: 2.11s
253:	learn: 0.4063901	total: 11.4s	remaining: 2.07s
254:	learn: 0.4063030	total: 11.5s	remaining: 2.02s
255:	learn: 0.4062378	total: 11.5s	remaining: 1.98s
256:	learn: 0.4060813	total: 11.6s	remaining: 1.93s
257:	learn: 0.4059837	total: 11.6s	remaining: 1.89s
258:	learn: 0.4059003	total: 11.6s	remaining: 1.84s
259:	learn: 0.4058158	total: 11.7s	remaining: 1.8s
260:	learn: 0.4056743	total: 11.7s	remaining: 1.75s
261:	learn: 0.4055851	total: 11.8s	remaining: 1.71s
262:	learn: 0.4054445	total: 11.8s	remaining: 1.66s
263:	learn: 0.4053153	total: 11.9s	remaining: 1.62s
264:	learn: 0.4051942	total: 11.9s	remaining: 1.57s
265:	learn: 0.4051008	total: 11.9s	remaining: 1.53s
266:	learn: 0.

AttributeError: 'CatBoostClassifier' object has no attribute 'CatBoostClassifier'

In [34]:
import catboost as cb
#With Categorical features
clf = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=255, \
                            depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(train,y_train, cat_features= cat_features_index)
auc(clf, train, test)



0:	total: 211ms	remaining: 1m 45s
1:	total: 325ms	remaining: 1m 21s
2:	total: 578ms	remaining: 1m 35s
3:	total: 758ms	remaining: 1m 33s
4:	total: 956ms	remaining: 1m 34s
5:	total: 1.18s	remaining: 1m 36s
6:	total: 1.36s	remaining: 1m 35s
7:	total: 1.45s	remaining: 1m 29s
8:	total: 1.65s	remaining: 1m 30s
9:	total: 1.86s	remaining: 1m 31s
10:	total: 2.1s	remaining: 1m 33s
11:	total: 2.36s	remaining: 1m 36s
12:	total: 2.57s	remaining: 1m 36s
13:	total: 2.82s	remaining: 1m 37s
14:	total: 3.07s	remaining: 1m 39s
15:	total: 3.31s	remaining: 1m 40s
16:	total: 3.52s	remaining: 1m 40s
17:	total: 3.79s	remaining: 1m 41s
18:	total: 4.01s	remaining: 1m 41s
19:	total: 4.26s	remaining: 1m 42s
20:	total: 4.47s	remaining: 1m 42s
21:	total: 4.73s	remaining: 1m 42s
22:	total: 4.96s	remaining: 1m 42s
23:	total: 5.2s	remaining: 1m 43s
24:	total: 5.41s	remaining: 1m 42s
25:	total: 5.67s	remaining: 1m 43s
26:	total: 5.91s	remaining: 1m 43s
27:	total: 6.13s	remaining: 1m 43s
28:	total: 6.34s	remaining: 1m 4

233:	total: 56.6s	remaining: 1m 4s
234:	total: 56.9s	remaining: 1m 4s
235:	total: 57.1s	remaining: 1m 3s
236:	total: 57.4s	remaining: 1m 3s
237:	total: 57.7s	remaining: 1m 3s
238:	total: 58s	remaining: 1m 3s
239:	total: 58.2s	remaining: 1m 3s
240:	total: 58.4s	remaining: 1m 2s
241:	total: 58.7s	remaining: 1m 2s
242:	total: 58.9s	remaining: 1m 2s
243:	total: 59.1s	remaining: 1m 2s
244:	total: 59.4s	remaining: 1m 1s
245:	total: 59.6s	remaining: 1m 1s
246:	total: 59.9s	remaining: 1m 1s
247:	total: 1m	remaining: 1m 1s
248:	total: 1m	remaining: 1m
249:	total: 1m	remaining: 1m
250:	total: 1m	remaining: 1m
251:	total: 1m 1s	remaining: 1m
252:	total: 1m 1s	remaining: 59.8s
253:	total: 1m 1s	remaining: 59.6s
254:	total: 1m 1s	remaining: 59.3s
255:	total: 1m 1s	remaining: 59.1s
256:	total: 1m 2s	remaining: 58.8s
257:	total: 1m 2s	remaining: 58.6s
258:	total: 1m 2s	remaining: 58.4s
259:	total: 1m 2s	remaining: 58.1s
260:	total: 1m 3s	remaining: 58s
261:	total: 1m 3s	remaining: 57.8s
262:	total: 1

465:	total: 1m 55s	remaining: 8.44s
466:	total: 1m 55s	remaining: 8.19s
467:	total: 1m 56s	remaining: 7.95s
468:	total: 1m 56s	remaining: 7.7s
469:	total: 1m 56s	remaining: 7.45s
470:	total: 1m 56s	remaining: 7.2s
471:	total: 1m 57s	remaining: 6.95s
472:	total: 1m 57s	remaining: 6.7s
473:	total: 1m 57s	remaining: 6.45s
474:	total: 1m 57s	remaining: 6.21s
475:	total: 1m 58s	remaining: 5.96s
476:	total: 1m 58s	remaining: 5.71s
477:	total: 1m 58s	remaining: 5.46s
478:	total: 1m 59s	remaining: 5.22s
479:	total: 1m 59s	remaining: 4.97s
480:	total: 1m 59s	remaining: 4.72s
481:	total: 1m 59s	remaining: 4.48s
482:	total: 2m	remaining: 4.23s
483:	total: 2m	remaining: 3.98s
484:	total: 2m	remaining: 3.74s
485:	total: 2m 1s	remaining: 3.49s
486:	total: 2m 1s	remaining: 3.25s
487:	total: 2m 1s	remaining: 3s
488:	total: 2m 2s	remaining: 2.75s
489:	total: 2m 2s	remaining: 2.5s
490:	total: 2m 2s	remaining: 2.25s
491:	total: 2m 2s	remaining: 2s
492:	total: 2m 3s	remaining: 1.75s
493:	total: 2m 3s	rema

(0.9166459886835678, 0.712508472765397)

In [37]:
clf.get_params()

{'iterations': 500,
 'learning_rate': 0.15,
 'depth': 10,
 'l2_leaf_reg': 9,
 'loss_function': 'Logloss',
 'one_hot_max_size': 255,
 'eval_metric': 'AUC'}

In [39]:
y_pred = clf.predict_proba(test_df)[:, 1]
write_to_submission_file(y_pred, 'subm2.csv')